In [0]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

Загружаем фичи

In [0]:
data_root = "/content/drive/My Drive/antifraud/agg"

In [0]:
interval = pd.DataFrame()
agg = pd.DataFrame()
count = pd.DataFrame()
mcc = pd.DataFrame()
time = pd.DataFrame()

In [4]:
for f in tqdm(os.listdir(data_root)):
  if "agg_features" in f:
    agg = pd.concat((agg, pd.read_csv(os.path.join(data_root, f))))
  elif "mcc_features" in f:
    mcc = pd.read_csv(os.path.join(data_root, f))
  elif "time_features" in f:
    time = pd.concat((time, pd.read_csv(os.path.join(data_root, f))))
  elif "intervals" in f:
    interval = pd.read_csv(os.path.join(data_root, f))
  elif "count_features" in f:
    count = pd.read_csv(os.path.join(data_root, f))

 94%|█████████▍| 106/113 [02:39<00:07,  1.10s/it]/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
100%|██████████| 113/113 [02:54<00:00,  1.54s/it]


In [0]:
agg = agg[agg.amount_original.str.isdigit().fillna(False)]

Берем таргет

In [0]:
path_root_agg = "/content/drive/My Drive/antifraud/trans_april_may_st3_agg"

In [0]:
tr_data = pd.DataFrame()

In [0]:
files = os.listdir(path_root_agg)
files.sort()

In [9]:
for f in tqdm(files[1:]):
  tr_data = tr_data.append(pd.read_parquet(os.path.join(path_root_agg, f)))

100%|██████████| 200/200 [10:21<00:00,  3.11s/it]


In [0]:
path_root_trans = "/content/drive/My Drive/antifraud/trans_april_may_st0_markup_incoming_transactions_anon"

In [0]:
data_trans = pd.DataFrame()

In [0]:
files = os.listdir(path_root_trans)
files.sort()

In [13]:
for f in tqdm(files[1:]):
  data_trans = data_trans.append(pd.read_parquet(os.path.join(path_root_trans, f)))

100%|██████████| 7/7 [00:19<00:00,  2.76s/it]


In [0]:
ids1 = set(tr_data.event_id)
ids2 = set(data_trans.event_id)
target_ids = ids1.intersection(ids2)

In [0]:
target = tr_data[tr_data.event_id.isin(target_ids)][["event_id", "USER_ID",	"USER_ID_RCVR",	"CLASS"]]

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
def tr_tst_split(tr):
    genue = tr[tr.CLASS == 0].sort_values("USER_ID_RCVR")
    fraud = tr[tr.CLASS == 1].sort_values("USER_ID_RCVR")
    
    fraud_tr, fraud_tst = train_test_split(fraud, test_size = 0.2, shuffle = False)
    genue_tr, genue_tst = train_test_split(genue, test_size = 0.2, shuffle = False)
    print(set(fraud_tr.USER_ID_RCVR).intersection(fraud_tst.USER_ID_RCVR))
    print(len(set(fraud_tr.USER_ID).intersection(fraud_tst.USER_ID)))
    
    return pd.concat([fraud_tr, genue_tr]), pd.concat([fraud_tst, genue_tst])

In [18]:
target_train, target_test = tr_tst_split(target)

set()
44


Объединим фичи в одну табличку

In [0]:
users_agg = tr_data[["USER_ID",	"USER_ID_RCVR"]].values.ravel()

In [0]:
users_trans = data_trans[["user_id", "user_id_rcvr"]].values.ravel()

In [0]:
users_hist = interval.user_id.values

In [22]:
users_agg

array(['100047377', '90581096', '100093853', ..., '9840857', 'VSP4488145',
       '71672720'], dtype=object)

In [23]:
users_trans

array(['1870825', '5540217', '23465860', ..., '59430976', '101199501',
       '56841199'], dtype=object)

In [24]:
users_hist

array([10000005, 100000436, 100000736, ..., 'VSP9970871', 'VSP9984110',
       'VSP9985663'], dtype=object)

In [25]:
len(set(users_hist).intersection(set(users_agg)))

80439

In [26]:
len(set(users_hist).intersection(set(users_trans)))

49288

In [0]:
a = set(users_trans).intersection(set(users_agg))

In [28]:
len(set(users_hist).intersection(a))

48715

In [0]:
interval.set_index("user_id", inplace=True)
time.set_index("user_id", inplace=True)
count.set_index("Unnamed: 0", inplace=True)
mcc.set_index("Unnamed: 0", inplace=True)

In [0]:
interval.index = interval.index.astype(str)
time.index = time.index.astype(str)
count.index = count.index.astype(str)
mcc.index = mcc.index.astype(str)

Бахнем тф идф

In [0]:
0-6 28-34 56-62

In [0]:
agg.set_index('Unnamed: 0', inplace=True)

In [0]:
col = list(agg.columns)

In [0]:
col = col[0:7]+col[28:35]+col[56:63]

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer

In [0]:
tfidf = TfidfTransformer()

In [0]:
to_convert = agg[col]

In [0]:
converted = tfidf.fit_transform(to_convert)

In [0]:
converted = pd.DataFrame(converted.toarray())

In [0]:
converted.set_index(agg.index, inplace = True)

In [0]:
agg.drop(col, inplace=True, axis = 1)

In [0]:
agg = pd.concat((agg, converted), axis = 1)

In [41]:
agg

,amount_original.7,amount_original.8,amount_original.9,amount_original.10,amount_original.11,amount_original.12,amount_original.13,amount_original.14,amount_original.15,amount_original.16,amount_original.17,amount_original.18,amount_original.19,amount_original.20,amount_original.21,amount_original.22,amount_original.23,amount_original.24,amount_original.25,amount_original.26,amount_original.27,amount_original.35,amount_original.36,amount_original.37,amount_original.38,amount_original.39,amount_original.40,amount_original.41,amount_original.42,amount_original.43,amount_original.44,amount_original.45,amount_original.46,amount_original.47,amount_original.48,amount_original.49,amount_original.50,amount_original.51,amount_original.52,amount_original.53,...,amount_original.65,amount_original.66,amount_original.67,amount_original.68,amount_original.69,amount_original.70,amount_original.71,amount_original.72,amount_original.73,amount_original.74,amount_original.75,amount_original.76,amount_original.77,amount_original.78,amount_original.79,amount_original.80,amount_original.81,amount_original.82,amount_original.83,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100017347,0.0,0.0,3006.375,441.80891472868257,0.0,1.0,13000.0,0.0,0.0,0.28865979381443296,0.6649484536082474,0.010309278350515464,0.005154639175257732,0.030927835051546393,0.0,0.0,0.1827674257631758,0.026859017263043954,0.0,6.079328951417432e-05,0.7903127636842662,0.0,0.0,3006.375,441.80891472868257,0.0,1.0,13000.0,0.0,0.0,0.28865979381443296,0.6649484536082474,0.010309278350515464,0.005154639175257732,0.030927835051546393,0.0,0.0,0.1827674257631758,0.026859017263043954,0.0,...,3006.375,441.80891472868257,0.0,1.0,13000.0,0.0,0.0,0.28865979381443296,0.6649484536082474,0.010309278350515464,0.005154639175257732,0.030927835051546393,0.0,0.0,0.1827674257631758,0.026859017263043954,0.0,6.079328951417432e-05,0.7903127636842662,0.000000,0.000000,0.231685,0.527187,0.029687,0.007248,0.028192,0.000000,0.000000,0.231685,0.527187,0.029687,0.007248,0.028192,0.000000,0.000000,0.231685,0.527187,0.029687,0.007248,0.028192
10003418,0.0,0.0,2015.0714285714287,1582.8951948051952,0.0,0.0,20960.0,0.0,0.0,0.14583333333333334,0.8020833333333334,0.0,0.0,0.052083333333333336,0.0,0.0,0.0820536756757903,0.0644554664920199,0.0,0.0,0.8534908578321897,0.0,0.0,2015.0714285714287,1582.8951948051952,0.0,0.0,20960.0,0.0,0.0,0.14583333333333334,0.8020833333333334,0.0,0.0,0.052083333333333336,0.0,0.0,0.0820536756757903,0.0644554664920199,0.0,...,2015.0714285714287,1582.8951948051952,0.0,0.0,20960.0,0.0,0.0,0.14583333333333334,0.8020833333333334,0.0,0.0,0.052083333333333336,0.0,0.0,0.0820536756757903,0.0644554664920199,0.0,0.0,0.8534908578321897,0.000000,0.000000,0.104234,0.566287,0.000000,0.000000,0.042278,0.000000,0.000000,0.104234,0.566287,0.000000,0.000000,0.042278,0.000000,0.000000,0.104234,0.566287,0.000000,0.000000,0.042278
100115786,0.0,0.0,2070.444476744186,721.8241794871794,0.0,1.0,3568.181818181818,0.0,0.01015228426395939,0.2910321489001692,0.6598984771573604,0.0,0.001692047377326565,0.03722504230118443,0.0,0.0,0.3254673576524504,0.11346848999146922,0.0,0.00015719685377134775,0.560906955502309,0.0,0.0,2070.444476744186,721.8241794871794,0.0,1.0,3568.181818181818,0.0,0.01015228426395939,0.2910321489001692,0.6598984771573604,0.0,0.001692047377326565,0.03722504230118443,0.0,0.0,0.3254673576524504,0.11346848999146922,0.0,...,2070.444476744186,721.8241794871794,0.0,1.0,3568.181818181818,0.0,0.01015228426395939,0.2910321489001692,0.6598984771573604,0.0,0.001692047377326565,0.03722504230118443,0.0,0.0,0.3254673576524504,0.11346848999146922,0.0,0.00015719685377134775,0.560906955502309,0.000000,0.023326,0.234773,0.525835,0.000000,0.002391,0.034104,0.000000,0.023326,0.234773,0.525835,0.000000,0.002391,0.034104,0.000000,0.023326,0.234773,0.525835,0.000000,0.002391,0.034104
10021523,0.0,0.0,17594.085714285713,5630.3491489

Склеим все таблицы

In [65]:
agg.index

Index(['100017347', '10003418', '100115786', '10021523', '100224646', '100234',
       '100251803', '100259027', '100307156', '100309774',
       ...
       'VSP4147568', 'VSP4181016', 'VSP444210', 'VSP4561513', 'VSP4562900',
       'VSP4582415', 'VSP4900548', 'VSP752226', 'VSP82951', 'VSP960095'],
      dtype='object', name='Unnamed: 0', length=605851)

In [67]:
mcc.index.astype(str)

Index(['10000005', '100000436', '100000736', '100000741', '100000799',
       '100000920', '100001505', '100001624', '100001632', '100002060',
       ...
       'VSP9943592', 'VSP9943981', 'VSP9944235', 'VSP9944444', 'VSP9944862',
       'VSP9970837', 'VSP9970855', 'VSP9970871', 'VSP9984110', 'VSP9985663'],
      dtype='object', name='Unnamed: 0', length=605852)

In [0]:
ind1 = set(agg.index.astype(str))
ind2 = set(mcc.index.astype(str))
ind3 = set(interval.index.astype(str))
ind4 = set(time.index.astype(str))
ind5 = set(count.index.astype(str))

ind = set.intersection(ind1, ind2, ind3, ind4, ind5)

In [71]:
len(ind)

366016

In [0]:
data = pd.concat((agg.loc[ind], mcc.loc[ind].drop_duplicates(), interval.loc[ind], time.loc[ind], count.loc[ind]), axis = 1)

In [89]:
data

,amount_original.7,amount_original.8,amount_original.9,amount_original.10,amount_original.11,amount_original.12,amount_original.13,amount_original.14,amount_original.15,amount_original.16,amount_original.17,amount_original.18,amount_original.19,amount_original.20,amount_original.21,amount_original.22,amount_original.23,amount_original.24,amount_original.25,amount_original.26,amount_original.27,amount_original.35,amount_original.36,amount_original.37,amount_original.38,amount_original.39,amount_original.40,amount_original.41,amount_original.42,amount_original.43,amount_original.44,amount_original.45,amount_original.46,amount_original.47,amount_original.48,amount_original.49,amount_original.50,amount_original.51,amount_original.52,amount_original.53,...,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,fraud interval,genue interval,mu,std,get_tr,get_amount,completed_tr,taked_amount,ratio count,ratio amount
54317447,0.0,0.0,4112.04,420.6125806451611,0.0,0.0,25000.0,0.0,0.0,0.11682242990654206,0.8691588785046729,0.0,0.009345794392523364,0.004672897196261682,0.0,0.0,0.13923706950370285,0.014242289259205192,0.0,0.0,0.8465206412370919,0.0,0.0,4112.04,420.6125806451611,0.0,0.0,25000.0,0.0,0.0,0.11682242990654206,0.8691588785046729,0.0,0.009345794392523364,0.004672897196261682,0.0,0.0,0.13923706950370285,0.014242289259205192,0.0,...,0.388920,0.263142,0.180883,0.157575,-0.102024,0.101540,-0.749146,-0.367239,0.018859,-0.440355,0.228761,-0.286195,0.266185,-0.658375,-0.185313,0.752869,0.139155,-0.250206,0.398385,-0.386434,-0.119445,0.222525,-0.215487,-0.079087,-0.137517,-0.034757,0.171067,0.086892,-0.074150,0.274457,0.920561,0.906542,0.536770,1.065243,0.0,0.0,1.0,25000.0,1.0,1.0
103027886,0.0,0.0,2194.9298245614036,1431.0404999999998,0.0,0.5,16216.666666666666,0.0,0.0,0.34545454545454546,0.6060606060606061,0.0,0.012121212121212121,0.03636363636363636,0.0,0.0,0.11061405389337896,0.07211765461441962,0.0,2.519762879332193e-05,0.817243093863408,0.0,0.0,2194.9298245614036,1431.0404999999998,0.0,0.5,16216.666666666666,0.0,0.0,0.34545454545454546,0.6060606060606061,0.0,0.012121212121212121,0.03636363636363636,0.0,0.0,0.11061405389337896,0.07211765461441962,0.0,...,-0.118896,0.029270,-0.086358,-0.150298,-0.272265,0.008505,-0.237279,0.006163,-0.000561,0.023639,0.322057,-0.090644,0.024356,0.024158,0.152650,-0.149981,0.145746,-0.221846,0.075736,-0.041426,0.111754,-0.447907,-0.055451,-0.023676,-0.340498,0.059906,0.125347,0.037581,-0.018314,0.007113,0.842424,0.769697,1.101873,1.078853,0.0,0.0,1.0,69750.0,1.0,1.0
48392221,0.0,0.0,1755.6666666666667,906.707,0.0,0.0,4522.727272727273,0.0,0.0,0.11842105263157894,0.5263157894736842,0.013157894736842105,0.05263157894736842,0.2894736842105263,0.0,0.0,0.24434822579051427,0.12619266001243962,0.0,0.0,0.629459114197046,0.0,0.0,1755.6666666666667,906.707,0.0,0.0,4522.727272727273,0.0,0.0,0.11842105263157894,0.5263157894736842,0.013157894736842105,0.05263157894736842,0.2894736842105263,0.0,0.0,0.24434822579051427,0.12619266001243962,0.0,...,-0.041184,0.001271,-0.030232,0.055662,-0.285814,0.029964,-0.111976,-0.006946,0.007711,-0.008219,0.108887,0.023119,0.032767,-0.091959,0.044429,-0.015617,-0.037901,-0.072167,0.183816,-0.063037,-0.091515,-0.137109,-0.090385,-0.025339,-0.022365,0.065750,0.029587,-0.128797,-0.166793,-0.023091,0.881579,0.934211,-0.845424,0.781826,1.0,1000.0,0.0,0.0,0.0,0.0
107965734,0.0,0.0,577.3518518518518,313.84072463768115,0.0,0.0,444.44444444444446,0.0,0.0,0.2634146341463415,0.6731707317073171,0.0,0.01951219512195122,0.04390243902439024,0.0,0.0,0.43226703273627753,0.23497456248870685,0.0,0.0,0.3327584047750156,0.0,0.0,577.3518518518518,313.84072463768115,0.0,0.0,444.44444444444446,0.0,0.0,0.2634146341463415,0.6731707317073171,0.0,0.01951219512195122,0.04390243902439024,0.0,0.0,0.43226703273627753,0.23497456248870685,0.0,...,-0.325472,0.160281,-0.349490,-0.068562,-0.309931,0.174384,-0.436787,-0.292087,0.055928,0.433872,-0.044263,0.139909,-0.023532,0.008612,0

In [0]:
data.to_csv("/content/drive/My Drive/antifraud/agg/training1.csv")

Теперь другие таблички

In [91]:
target

,event_id,USER_ID,USER_ID_RCVR,CLASS
0,845f9da6751a47a28407ae934fe3c7da,100047377,90581096,0
1,d268a6452b4f47fb9c6e161dbd4b1d9d,100093853,51785703,0
2,32e2ca3fad5f4ae89aa51be508be0a03,100241756,14041952,0
5,80608b0f3a3b4e20acad3e923e632119,100445075,103157424,0
6,09d73e640c6f4ab99d25fd04986b09b2,100445075,60381182,0
...,...,...,...,...
1976,bd138f29ec544e7b9bf2178246ab0e0f,VSP25644210,VSP20227278,0
1977,8db26b07f03743ed940ca6ef3aba32ba,VSP297074,53826758,0
1978,87c21609ccbe479eba9e49dd3af25722,VSP32130102,54264326,0
1979,e7ccd23f1b2d4060926a9bf1422bb9e0,VSP32372012,47093731,0


In [92]:
data_trans

,event_id,user_id,user_id_rcvr,policy_action,risk_score,amount_original,event_time_
0,83cfb3a65bd943f18c4da6dc653ccf68,1870825,5540217,REVIEW,684,120000,1.554394e+09
1,a9cc86a4e52a4d898e6f593f50515be0,23465860,26903133,REVIEW,967,16000,1.554463e+09
2,b855e9868024450593239d34d4610513,MBK9395053,MBK9836211,REVIEW,994,10000,1.554536e+09
3,4514599aa3f54aa497311dc9289dfd2d,37570001,37140634,REVIEW,733,340000,1.554549e+09
4,63592180fc8b4a55a3c0fa12d0f79fa4,29705334,53214797,REVIEW,977,10000,1.554810e+09
...,...,...,...,...,...,...,...
14362,fb661f33d21f471fa936599cedbd47ab,49194968,11585469,REVIEW,536,15000,1.555313e+09
14363,d747ed5790554244b2f65972fd5c0059,2026042,22818226,REVIEW,650,8000,1.555915e+09
14364,6bb49760d44c48bfbbfc5378e5167da6,MBK5632283,19364183,REVIEW,644,25000,1.554291e+09
14365,f3d48a2d15c14effb267678b16eba53d,28570186,59430976,REVIEW,796,179000,1.556518e+09


In [0]:
ids = set(target.event_id).intersection(set(data_trans.event_id))

In [0]:
data_trans.set_index('event_id', inplace=True)
target.set_index('event_id', inplace=True)
data_trans = data_trans.loc[ids]

data_trans['class'] = target.loc[ids].CLASS

In [96]:
data_trans

,user_id,user_id_rcvr,policy_action,risk_score,amount_original,event_time_,class
event_id,,,,,,,
e275d34e8a014b23b77ce35a78baa5c1,76452782,107639702,REVIEW,781,200000,1.554961e+09,0
b79338121841411282560b5523ebf437,456859,68271399,REVIEW,948,70000,1.555665e+09,0
3b927f7fab9b4072bb5c210493c6e383,85694814,15767119,REVIEW,660,50000,1.555667e+09,0
33eb539caa8f4f8eba86f228f2eebf41,19687752,110493665,REVIEW,679,30000,1.555593e+09,0
a01144ba86b74730a5e5fab90dc30773,44145721,3009693,REVIEW,572,99000,1.554955e+09,0
...,...,...,...,...,...,...,...
d14479ddd9274be7ae5ec80e32975804,71652551,47664948,REVIEW,988,30000,1.556027e+09,0
9b5cdfb7008041daa872e593718eef7e,104140563,67648167,REVIEW,999,74151,1.555056e+09,0
a9f260d1b8744fe0a474c5b718b78bee,6213262,11179642,REVIEW,949,50000,1.555754e+09,0


In [97]:
(data_trans.user_id.isin(ind)&data_trans.user_id_rcvr.isin(ind)).sum()

219312

In [100]:
data_trans[(data_trans.user_id.isin(ind)&data_trans.user_id_rcvr.isin(ind))]["class"].sum()

4885

In [0]:
data_trans = data_trans[(data_trans.user_id.isin(ind)&data_trans.user_id_rcvr.isin(ind))]

In [0]:
X1 = data.loc[data_trans.user_id].set_index(data_trans.index)
X2 = data.loc[data_trans.user_id_rcvr].set_index(data_trans.index)

X = pd.concat((data_trans, X1, X2), axis = 1)

In [104]:
X

,user_id,user_id_rcvr,policy_action,risk_score,amount_original,event_time_,class,amount_original.7,amount_original.8,amount_original.9,amount_original.10,amount_original.11,amount_original.12,amount_original.13,amount_original.14,amount_original.15,amount_original.16,amount_original.17,amount_original.18,amount_original.19,amount_original.20,amount_original.21,amount_original.22,amount_original.23,amount_original.24,amount_original.25,amount_original.26,amount_original.27,amount_original.35,amount_original.36,amount_original.37,amount_original.38,amount_original.39,amount_original.40,amount_original.41,amount_original.42,amount_original.43,amount_original.44,amount_original.45,amount_original.46,...,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,fraud interval,genue interval,mu,std,get_tr,get_amount,completed_tr,taked_amount,ratio count,ratio amount
event_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
e275d34e8a014b23b77ce35a78baa5c1,76452782,107639702,REVIEW,781,200000,1.554961e+09,0,0.0,0.0,23754.450495049507,12522.955726141076,0.0,0.16666666666666666,59716.666666666664,0.0,0.0,0.41140529531568226,0.4908350305498982,0.0,0.024439918533604887,0.07331975560081466,0.0,0.0,0.24745704122753304,0.13045528340300194,0.0,1.7362152920853278e-06,0.622085939154173,0.0,0.0,23754.450495049507,12522.955726141076,0.0,0.16666666666666666,59716.666666666664,0.0,0.0,0.41140529531568226,0.4908350305498982,0.0,...,-0.237563,-0.132006,0.041838,0.311601,-0.291238,0.059075,-0.167444,0.135174,-0.069935,0.102377,0.330835,0.092129,-0.431354,0.226632,0.144823,0.272880,-0.168493,-0.363585,0.136320,-0.044881,0.267745,-0.201858,0.196467,0.281289,-0.054344,-0.480670,0.135495,-0.072643,-0.024817,-0.070464,0.832230,0.834437,-0.180305,1.390409,2.0,234000.0,1.0,200000.0,0.333333,0.460829
b79338121841411282560b5523ebf437,456859,68271399,REVIEW,948,70000,1.555665e+09,0,0.0,0.0,18231.365,14702.134742268041,0.0,0.0,137142.85714285713,0.0,0.0,0.09230769230769231,0.7461538461538462,0.0,0.0,0.16153846153846155,0.0,0.0,0.10719517594273277,0.08644431837282544,0.0,0.0,0.8063605056844418,0.0,0.0,18231.365,14702.134742268041,0.0,0.0,137142.85714285713,0.0,0.0,0.09230769230769231,0.7461538461538462,0.0,...,-0.011117,0.122384,0.060656,-0.030050,0.113085,-0.211656,-0.059596,0.071888,-0.044135,-0.140813,-0.195256,0.132398,0.060282,0.237854,0.096108,-0.193738,-0.057542,-0.050599,0.223736,-0.106325,0.047490,-0.183792,0.079057,-0.043384,-0.084015,-0.110875,0.044955,0.061263,0.020861,-0.102460,0.861111,0.916667,-0.326525,0.889272,0.0,0.0,1.0,70000.0,1.000000,1.000000
3b927f7fab9b4072bb5c210493c6e383,85694814,15767119,REVIEW,660,50000,1.555667e+09,0,0.0,0.0,36316.836153846154,4497.702105263158,0.0,0.0,0.0,0.0,0.0,0.18571428571428572,0.8142857142857143,0.0,0.0,0.0,0.0,0.0,0.8898014703312409,0.11019852966875902,0.0,0.0,0.0,0.0,0.0,36316.836153846154,4497.702105263158,0.0,0.0,0.0,0.0,0.0,0.18571428571428572,0.8142857142857143,0.0,...,-0.278087,0.023665,0.139937,0.091826,-0.100162,0.623462,-0.237048,0.081282,-0.588624,0.433243,-0.432395,-0.042898,-0.102323,0.229450,-0.255163,0.400021,0.504427,-0.573631,0.342428,-0.326198,-0.396140,0.006326,0.094722,0.384923,-0.252259,-0.151402,0.242314,0.058323,-0.202043,0.261761,0.892135,0.923596,-0.099400,1.203622,0.0,0.0,1.0,50000.0,1.000000,1.000000
33eb539caa8f4f8eba86f228f2eebf41,19687752,110493665,REVIEW,679,30000,1.555593e+09,0,0.0,0.0,13791.203007518798,2623.7284431137714,0.0,0.3333333333333333,8757.777777777777,0.0,0.0,0.15665488810365136,0.7868080094228505,0.0,0.0035335689045936395,0.053003533568904596,0.0,0.0,0.5478560239070089,0.10422770456445102,0.0,1.3241678375418601e-05,0.3479030298501647,0.0,0.0,13791.203007518798,2623.7284431137714,0.0,0.3333333333333333,8757.777777777777,0.0,0.0,0.15665488810365136,0.7868080094228505,0.0,...,-0.011393,0.048864,-0.031963,-0.037061,-0.065860,-0.029486,-0.117516,-0.013139,0.043052,-0.034881,-0.102470,-0.074834,-0.007938,-0.0

In [0]:
X.to_csv("/content/drive/My Drive/antifraud/agg/train_data.csv")